In [28]:
import os
from langchain.llms.openai import OpenAI
# from langchain.sql_database import SQLDatabase
from dotenv import load_dotenv # pip install load-dotenv
load_dotenv()

# import psycopg2 # pip install psycopg-binary
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent

PASSWORD = os.getenv("PG_PASSWORD")
USERNAME = os.getenv("PG_USERNAME")
HOST =  os.getenv("PG_HOST")
DATABASE =  os.getenv("PG_DATABASE")
PORT =  os.getenv("PG_PORT")
CONNECTION_STRING = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

In [ ]:
# db
db = SQLDatabase.from_uri(CONNECTION_STRING)
db 

In [ ]:
# GPT
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm

In [ ]:
db.get_usable_table_names()
db.run("SELECT login FROM public.res_users LIMIT 10;")

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool! 

You have access to the following tables: {table_names}

If the question does not seem related to the database, just return "I don't know" as the answer."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}"), MessagesPlaceholder("agent_scratchpad")]
)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True, prompt=prompt)
agent_executor

In [ ]:
query = "login"

agent_executor.invoke(
    {
        "input":  f"Find all name of {query} of res_users, and count how many there are"
    }
)

In [ ]:
examples = [
    {"input": "List all user's login.", "query": "SELECT login FROM res_users;"},
    {
        "input": "Find all knowledges.",
        "query": "SELECT * FROM SELECT * FROM public.knowledge_article;",
    },
]